### 🔍 **Framework Project: Change Background Image using U-Net**

---

## 📌 1. Giới thiệu

Trong lĩnh vực xử lý ảnh, đặc biệt là bài toán **phân đoạn ảnh (image segmentation)**, việc tách tiền cảnh (foreground) khỏi hậu cảnh (background) là một bước quan trọng. Kiến trúc **U-Net**, được giới thiệu lần đầu trong lĩnh vực ảnh sinh học, đã chứng minh hiệu quả vượt trội cho các bài toán segmentation mà không cần một lượng lớn dữ liệu huấn luyện.

---

## 🧠 2. Kiến trúc U-Net

Kiến trúc U-Net gồm ba thành phần chính:

* **Encoder (Contracting Path)**: Trích xuất đặc trưng với các lớp tích chập (convolution) và giảm kích thước (max pooling).
* **Decoder (Expanding Path)**: Tăng kích thước ảnh đầu ra bằng các lớp up-convolution (deconvolution) để tái tạo lại phân vùng ảnh.
* **Skip Connections**: Giúp giữ lại thông tin chi tiết (ví dụ: biên ảnh) bằng cách kết nối trực tiếp encoder với decoder tại các tầng tương ứng.
* **1x1 Convolution cuối cùng**: Dự đoán phân lớp trên từng pixel.

```
Encoder (↓):        Input -> Conv -> ReLU -> Conv -> ReLU -> MaxPool
Decoder (↑):        UpConv -> Concatenate (skip) -> Conv -> ReLU -> Conv -> ReLU
```

### 🔧 Cấu trúc tổng quát


![Ảnh minh hoạ](https://media.geeksforgeeks.org/wp-content/uploads/20220614121231/Group14.jpg)


---

### 🧱 Kiến trúc chi tiết

| Thành phần     | Chi tiết                                          |
| -------------- | ------------------------------------------------- |
| Conv layers    | 3x3 padding + ReLU                                |
| MaxPooling     | 2x2                                               |
| Up-convolution | 2x2                                               |
| Output layer   | 1x1 convolution để dự đoán phân lớp tại mỗi pixel |

U-Net sử dụng **data augmentation mạnh mẽ** (flip, rotation, shift, elastic deformation) để tạo ra nhiều mẫu từ ảnh gốc – điều này giúp giải quyết vấn đề thiếu dữ liệu.

---

## 🛠️ 3. Ứng dụng U-Net cho Project Change Background

### 🎯 Mục tiêu

Sử dụng U-Net để **tách lớp foreground (người, vật thể chính)** khỏi ảnh, sau đó **thay thế background** bằng một ảnh mới.

---

### 🔧 Framework tổng quan

---

### 🚶‍♂️ Bước 1: Chuẩn bị Dữ liệu

* **Input**: Hình ảnh RGB chứa người hoặc vật thể chính cần giữ lại.
* **Resize** ảnh về kích thước cố định (thường là `256x256`).
* **Normalize** pixel trong khoảng `[0, 1]`.

---

### 🧠 Bước 2: Dự đoán Mask bằng U-Net

* Mô hình U-Net được **load từ trọng số đã huấn luyện** hoặc **huấn luyện lại** từ đầu (nếu dữ liệu đủ).
* Output là một **binary mask**, nơi `1` tương ứng với foreground (người), `0` là background.

---

### 🧪 Bước 3: Hậu xử lý Mask

* **Làm mịn mask** bằng các kỹ thuật như:

  * Morphological operation (erosion/dilation)
  * Gaussian blur
  * Thresholding

> Mục đích: Tăng độ chính xác biên và giảm nhiễu.

---

### ✂️ Bước 4: Tách foreground

Sử dụng mask để giữ lại vùng foreground:

```python
result = input_image * mask[..., np.newaxis]
```

---

### 🌄 Bước 5: Chèn nền mới

* Load ảnh nền mới (có cùng kích thước).
* Áp dụng công thức:

```python
output = result + background * (1 - mask[..., np.newaxis])
```

* Kết quả là một ảnh mới với foreground giữ nguyên và background đã thay đổi.

---

## 🧪 4. Đánh giá

Nhóm chạy cùng một mô hình với dữ liệu đầu vào giống nhau cho cả phiên bản tuần tự và phiên bản song song (dùng multiprocessing). Sau đó, nhóm so sánh độ chính xác (accuracy) của hai mô hình.

Ngoài ra, nhóm có thể sử dụng các tiêu chí đánh giá phổ biến:

| Chỉ số             | Giải thích                                          |
| ------------------ | --------------------------------------------------- |
| **IoU**            | Mức độ chồng lấp giữa mask dự đoán và ground truth  |
| **Dice Score**     | Tỷ lệ chính xác giữa vùng foreground được phân đoạn |
| **Pixel Accuracy** | Tỷ lệ pixel được phân loại đúng                     |

## 📌 5. Vì sao lại lựa chọn U-Net?

U-Net là một mô hình mạnh mẽ và đơn giản cho bài toán segmentation. Khi áp dụng vào project thay background ảnh:

* Mô hình có thể tách chính xác foreground, giúp thay background mượt mà.
* Có thể kết hợp thêm các kỹ thuật xử lý ảnh cổ điển để cải thiện đầu ra.
* U-Net pretrained có thể dùng với các tập dữ liệu segmentation thông dụng như COCO, Pascal VOC, hoặc tự thu thập nếu phù hợp.

---

## 📚 Tham khảo

* Ronneberger et al., *U-Net: Convolutional Networks for Biomedical Image Segmentation*, [arXiv:1505.04597](https://arxiv.org/pdf/1505.04597v1)
* [https://towardsdatascience.com/understanding-u-net-61276b10f360/](https://towardsdatascience.com/understanding-u-net-61276b10f360/)
* [https://github.com/milesial/Pytorch-UNet](https://github.com/milesial/Pytorch-UNet)
